In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# A Backtracking Solver for CSPs

In [2]:
interface CSP<V extends string, D extends string | number> {
 readonly variables: readonly V[];
 readonly values: readonly D[]; 
 readonly constraints: readonly string[];
}
type Assignment<V extends string, D extends string | number> = Partial<Record<V, D>>;
type Solution<V extends string, D extends string | number> = Record<V, D>;
type VisualizationCallback<V extends string, D extends string | number> = (
 assignment: Assignment<V, D>
) => void;
interface SolveResult<V extends string, D extends string | number> {
 readonly steps: number;
 readonly solution: Solution<V, D>;
}
interface AnnotatedConstraint {
  readonly formula: string;
  readonly vars: ReadonlySet<string>;
}

The function `collectVariables(expr)` takes a string `expr` that can be interpreted as a valid expression as its input and collects all variables occurring in `expr`.
It takes care to remove names that correspond to predefined values or functions. To achieve this, it defines a set of `builtIns` containing reserved names (like `Math`, `true`, `false`, `abs`, `min`) that are not treated as variables, ensuring that only true variable names are returned by `collectVariables`.

In [3]:
function collectVariables(expr: string): Set<string> {
  const identifierRegex = /[a-zA-Z_][a-zA-Z0-9_]*/g;
  const builtIns = new Set(['abs', 'min', 'max', 'pow', 'sum', 'len', 'Math', 'true', 'false']);
  const variables = new Set<string>();
  let match: RegExpExecArray | null;
  
  while ((match = identifierRegex.exec(expr)) !== null) {
    const candidate = match[0];
    if (!builtIns.has(candidate)) {
      variables.add(candidate);
    }
  } 
  return variables;
}

In [4]:
const expr = 'abs(x - y) + abs(z1 - z2)';
collectVariables(expr);

Set(4) { 'x', 'y', 'z1', 'z2' }


## The Backtracking Solver with Animation

The function `isConsistent(variable, value, assignment, constraints)` takes four arguments:
- `variable` is a variable that does not occur in `assignment`,
- `value` is a value that can be substituted for this variable,
- `assignment` is a *consistent* partial variable assignment.
- `constraints` is a set of pairs of the form $\langle f, V \rangle$ where $f$ is a formula and $V$ is the set of variables occurring in $f$.

This function returns `true` iff the partial variable assignment
$$\texttt{assignment} \cup \bigl\{\langle\texttt{variable} \mapsto\texttt{value}\rangle\bigr\}$$
is consistent with all the constraints $f$ occurring in `constraints`.

**Note** that the function `evaluateExpression` handles the variable substitution dynamically.

In [6]:
function isSubset<T>(subset: ReadonlySet<T>, superset: Set<T>): boolean {
  for (const elem of subset) {
    if (!superset.has(elem)) {
      return false;
    }
  }
  return true;
}

function evaluateExpression<V extends string, D extends string | number>(
  expr: string,
  context: Assignment<V, D>
): boolean {
  const jsExpr = expr
    .replace(/\band\b/g, '&&')
    .replace(/\bor\b/g, '||')
    .replace(/\bnot\b/g, '!');
  
  const argNames = Object.keys(context);
  const argValues = Object.values(context);
  
  try {
    const func = new Function(...argNames, `return (${jsExpr});`);
    const result: unknown = func(...argValues);
    
    return typeof result === 'boolean' ? result : false;
  } catch (e) {
    return false;
  }
}

In [7]:
function isConsistent<V extends string, D extends string | number>(
  variable: V,
  value: D,
  assignment: Assignment<V, D>,
  constraints: readonly AnnotatedConstraint[]
): boolean {
  const newAssignment: Assignment<V, D> = { ...assignment, [variable]: value };
  const assignedVars = new Set(Object.keys(newAssignment));
  
  for (const { formula, vars } of constraints) {
    if (vars.has(variable) && isSubset(vars, assignedVars)) {
      if (!evaluateExpression(formula, newAssignment)) {
        return false;
      }
    }
  }
  
  return true;
}

TYPE GUARD: Prüft zur Laufzeit, ob alle Variablen gesetzt sind.
Wenn true, verengt TypeScript den Typ von 'Assignment' auf 'Solution'.

In [9]:
function isComplete<V extends string, D extends string | number>(
  assignment: Assignment<V, D>,
  variables: readonly V[]
): assignment is Solution<V, D> {
  if (Object.keys(assignment).length !== variables.length) {
    return false;
  }
  for (const v of variables) {
    if (!(v in assignment)) {
      return false;
    }
  }
  return true;
}

The function `backtrackSearch` takes four arguments:
- `assignment` is a partial variable assignment that is represented as a dictionary. Initially, this assignment will be the empty dictionary. Every recursive call of `backtrackSearch` adds the assignment of one variable to the given assignment. The important invariant of recursive calls of `backtrackSearch` is that `assignment` is *consistent*, i.e. all constraints $f$ that contain only variables from the set $\mathtt{dom}(\mathtt{assignment})$ are satisfied.
- `csp` is an *augmented constraint satisfaction problem*,
  i.e. `csp` is a triple of the form
  $$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
  where
  - $\mathtt{Vars}$ is a set of strings which serve as *variables*,
  - $\mathtt{Values}$ is a set of *values* that can be assigned
    to the variables in $\mathtt{Vars}$.
  - $\mathtt{Constraints}$ is a set of pairs of the form $(f, V)$ where $f$ is a Boolean expression, while $V$ is the set of variables occurring in $f$.
- `state` is an object tracking the number of steps.
- `onUpdate` (optional) is a callback function for visualization.

The function `backtrackSearch` tries to find a solution of `csp` by recursively augmenting `assignment`.

In [10]:
function backtrackSearch<V extends string, D extends string | number>(
  assignment: Assignment<V, D>,
  variables: readonly V[],
  values: readonly D[],
  constraints: readonly AnnotatedConstraint[],
  state: { steps: number },
  onUpdate: VisualizationCallback<V, D> | undefined
): Solution<V, D> | null {
  
  if (onUpdate !== undefined) {
    onUpdate(assignment);   
  }
  
  if (isComplete(assignment, variables)) {
    return assignment; // TypeScript weiß jetzt sicher: Es ist 'Solution<V, D>'
  }
  
  const unassignedVar = variables.find(v => !(v in assignment));
  
  if (unassignedVar === undefined) {
    return null;
  }
  
  for (const value of values) {
    state.steps++;
    
    if (isConsistent(unassignedVar, value, assignment, constraints)) {
      const newAssignment: Assignment<V, D> = { 
        ...assignment, 
        [unassignedVar]: value 
      };
      
      const result = backtrackSearch(
        newAssignment,
        variables,
        values,
        constraints,
        state,
        onUpdate
      );
      
      if (result !== null) {
        return result;
      }
    }
  }
  
  return null;
}

The procedure `solve(csp)` takes a *constraint satisfaction problem*
`csp` as input. Here `csp` is a triple of the form
$$ \mathcal{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where
- `variables` is a set of strings which serve as *variables*,
- `values` is a set of *values* that can be assigned
  to the variables in the set `variables`.
- `constraints` is a set of formulas (constraints).

The main purpose of the function `solve` is to convert the CSP `csp` into an
*augmented CSP* where every constraint $f$ is annotated with
the variables occurring in $f$. This annotated CSP is then solved using the function
`backtrackSearch`.

In [11]:
function solve<V extends string, D extends string | number>(
  csp: CSP<V, D>,
  onUpdate?: VisualizationCallback<V, D>
): SolveResult<V, D> | null {
  const { variables, values, constraints } = csp;
  const state = { steps: 0 };
  
  const annotatedConstraints: AnnotatedConstraint[] = constraints.map(f => ({
    formula: f,
    vars: collectVariables(f)
  }));
  
  const solution = backtrackSearch(
    {},
    variables,
    values,
    annotatedConstraints,
    state,
    onUpdate
  );
  
  console.log(`Tested ${state.steps} partial assignments`);
  
  if (solution === null) {
    return null;
  }
  
  return {
    steps: state.steps,
    solution
  };
}